
#Developer Bastian - Unreal Lyra Attribute Code Creator

**Set a name, comment your attribute set and add a comman seperated list of attributes (no spaces allowed):**

In [ ]:
AttributSetName = 'AttributeSet_TestSet' #@param {type:"string"}
AttributSetComment = 'Test attributes with FText' #@param {type:"string"}
Attributes = 'Editorableness,CharCount,TextBeauty' #@param {type:"string"}
#@markdown To keep the number of C## classes at a single only (this is supposed to be a BP course after all...), I defined the base class (**UGameAttributeSet**) and logging category (**GameAttributeSet**) within this class.

#@markdown Only check the below box, if this is the **first** generated AttributeSet, for all further generated AttributeSets, uncheck the box:
FirstGeneratedAttributeSet = False #@param {type:"boolean"}
# set final vars from input
AttributSetName = AttributSetName.strip()
attributeList = Attributes.split(',')
attributeList = [s.strip() for s in attributeList]

# internal code fragments - .h
REP_FUNCTION = 'UFUNCTION()\n\tvoid OnRep_$ATTRIBUTE$(const FGameplayAttributeData& OldValue);'
UPROPERTY = 'UPROPERTY(BlueprintReadOnly, ReplicatedUsing = OnRep_$ATTRIBUTE$, Category = "$NAME$", Meta = (AllowPrivateAccess = true))\n\tFGameplayAttributeData $ATTRIBUTE$;'
GAMEPLAYTAGSDEF = 'UE_DECLARE_GAMEPLAY_TAG_EXTERN(TAG_$NAME$_$ATTRIBUTE$);'
# internal code fragments - .cpp
INIALIZE_ATTRIBUTES = '$ATTRIBUTE$(0.0f)'
OnRepFUNCTIONS = """void U$NAME$::OnRep_$ATTRIBUTE$(const FGameplayAttributeData& OldValue)
{
	static FProperty* Prop = FindFieldChecked&lt;FProperty&gt;(U$NAME$::StaticClass(), GET_MEMBER_NAME_CHECKED(U$NAME$, $ATTRIBUTE$));
	ULyraAbilitySystemComponent* ASC = CastChecked&lt;ULyraAbilitySystemComponent&gt;(GetOuter(), ECastCheckedType::NullAllowed);
	if (IsValid(ASC))
	{
		ASC->SetBaseAttributeValueFromReplication(FGameplayAttribute(Prop), $ATTRIBUTE$, OldValue);
	}
	else
	{
		UE_LOG(GameAttributeSet, Error, TEXT("U$NAME$s::OnRep_$ATTRIBUTE$: No valid ASC found"));
	}
}
"""
DOREP = 'DOREPLIFETIME_CONDITION_NOTIFY(U$NAME$, $ATTRIBUTE$, COND_OwnerOnly, REPNOTIFY_Always);'
GAMEPLAYTAGSIMPL = 'UE_DEFINE_GAMEPLAY_TAG(TAG_$NAME$_$ATTRIBUTE$, "AttributeSets.$NAME$.$ATTRIBUTE$");\nFGameplayTag U$NAME$::GetGameplayTag_$ATTRIBUTE$() const\n{\n\treturn FGameplayTag::RequestGameplayTag(FName("AttributeSets.$NAME$.$ATTRIBUTE$"));\n}'
# if FirstGeneratedAttributeSet = true, the class infomation is filled, otherwise it can be empty as we only need one definiiton of it for further sets
BASECLASS_LOGGING_H = """#pragma region Baseclass UGameAttributeSet: CAN ONLY EXISTS ONCE IN YOUR PROJECT. IF YOU GENERATE MULTIPLE ATTRIBUTE SETS, DELETE THIS SECTION STARTING WITH THE SECOND SET (OR UNCHECK THE BOX ON THE GENRERATOR FORM ON THE NEXT SET)
/**
 * Base Class for classes generated by the AttributeSet Generator (https://bit.ly/AttributeGenerator)
 */
UCLASS(NotBlueprintable, Abstract)
class UGameAttributeSet : public ULyraAttributeSet
{
	GENERATED_BODY()
public:
	UGameAttributeSet() {}

#pragma region ADD-REMOVE AttributeSet
	/** Attaches this AttributeSet to a given ASC */
	UFUNCTION(BlueprintCallable, BlueprintAuthorityOnly, Category = "UGameAttributeSet")
		FORCEINLINE void AttachToASC(ULyraAbilitySystemComponent* AbilitySystemComponent);

	/** Removes this AttributeSet from a given ASC. This can be dangerous if a value has not been already replicated and cannot find its corresponding AttributSet - this will crash the game!!! */
	UFUNCTION(BlueprintCallable, BlueprintAuthorityOnly, Category = "UGameAttributeSet")
		FORCEINLINE void RemoveFromASC(ULyraAbilitySystemComponent* AbilitySystemComponent);

		/** Looks for an instance of the given AttributeSet class in the given AbilitySetComponent and returns all Attributes found in that instance with their current values */
	UFUNCTION(BlueprintCallable, Category = "UGameAttributeSet")
		FORCEINLINE TArray&lt;FGameplayAttribute&gt; GetAttributesForAttributeSetClass(ULyraAbilitySystemComponent* AbilitySystemComponent, TSubclassOf&lt;UAttributeSet&gt; AttributSetClass);
#pragma endregion
};
#pragma region Logging
/** Macro to create a log category for everything generated here */
DECLARE_LOG_CATEGORY_EXTERN(GameAttributeSet, Log, All);
#pragma endregion
#pragma endregion DELETE UNTIL HERE IF YOU USE MULTIPLE ATTRIBUTE SETS AND FORGOT TO UNCHECK THE BOX"""
BASECLASS_LOGGING_CPP = """#pragma region Logging and UGameAttributeSet: MUST ONLY EXISTS ONCE IN YOUR PROJECT. IF YOU GENERATE MULTIPLE ATTRIBUTE SETS, DELETE THIS REGION STARTING WITH THE SECOND SET ATTRIBUTESET
#pragma region Logging
DEFINE_LOG_CATEGORY(GameAttributeSet);
#pragma endregion
#pragma region UGameAttributeSet
void UGameAttributeSet::AttachToASC(ULyraAbilitySystemComponent* AbilitySystemComponent)
{
	if (IsValid(AbilitySystemComponent))
	{
		auto AttributeSet = AbilitySystemComponent->GetSet&lt;UGameAttributeSet&gt;();
		if (!IsValid(AttributeSet))	// only grant if AttributeSet was not found already on ASC
		{
			AbilitySystemComponent->AddAttributeSetSubobject(this);
			AbilitySystemComponent->ForceReplication();
			UE_LOG(GameAttributeSet, Log, TEXT("AttachToASC: Attached '%s' to ASC"), *GetNameSafe(this));
		}
		else
		{
			UE_LOG(GameAttributeSet, Warning, TEXT("AttachToASC: AttributeSet '%s' already granted to ASC - not attached again to ASC"), *GetNameSafe(this));
		}
	}
	else
	{
		UE_LOG(GameAttributeSet, Error, TEXT("AttachToASC: Attaching '%s' to ASC failed, ASC not valid"), *GetNameSafe(this));
	}
}

void UGameAttributeSet::RemoveFromASC(ULyraAbilitySystemComponent* AbilitySystemComponent)
{
	if (IsValid(AbilitySystemComponent))
	{
		AbilitySystemComponent->ForceReplication();
		AbilitySystemComponent->RemoveSpawnedAttribute(this);
		AbilitySystemComponent->ForceReplication();
		UE_LOG(GameAttributeSet, Log, TEXT("Removed '%s' from ASC"), *GetNameSafe(this));
	}
	else
	{
		UE_LOG(GameAttributeSet, Error, TEXT("Removing '%s' from ASC failed, ASC not valid"), *GetNameSafe(this));
	}
}

TArray&lt;FGameplayAttribute&gt; UGameAttributeSet::GetAttributesForAttributeSetClass(ULyraAbilitySystemComponent* AbilitySystemComponent, TSubclassOf&lt;UAttributeSet&gt; AttributSetClass)
{
	TArray&lt;FGameplayAttribute&gt; AttributeArray;
	if (IsValid(AbilitySystemComponent))
	{
		const UAttributeSet* AttributeSet = AbilitySystemComponent->GetAttributeSet(AttributSetClass);
		if (IsValid(AttributeSet))
		{
			TArray&lt;FGameplayAttribute&gt; Attributes;
			AbilitySystemComponent->GetAllAttributes(Attributes);
			FString AttributeNames = "";
			for (FGameplayAttribute Attribute : Attributes)
			{
				UClass* AttributeClass = Attribute.GetAttributeSetClass();
				if (AttributeClass->IsChildOf(AttributSetClass))
				{
					AttributeArray.Add(Attribute);
					AttributeNames.Append(Attribute.AttributeName + ": " + FString::SanitizeFloat(Attribute.GetNumericValue(AttributeSet)) + "\\t");
				}
			}
			UE_LOG(GameAttributeSet, Log, TEXT("GetAttributesFromAttributeSet: Returned %d Attributes for AttributeSet class '%s': %s"), AttributeArray.Num(), *GetNameSafe(this), *AttributeNames);
		}
		else
		{
			UE_LOG(GameAttributeSet, Warning, TEXT("GetAttributesFromAttributeSet: No valid AttributeSet of class '' found"), *GetNameSafe(AttributSetClass));
		}
	}
	else
	{
		UE_LOG(GameAttributeSet, Warning, TEXT("GetAttributesFromAttributeSet: ASC not valid"), *GetNameSafe(AbilitySystemComponent));
	}
	return AttributeArray;
}
#pragma endregion
#pragma endregion DELETE UNTIL HERE IF YOU USE MULTIPLE ATTRIBUTE SETS"""
if not FirstGeneratedAttributeSet:
	BASECLASS_LOGGING_H = ""
	BASECLASS_LOGGING_CPP = ""
# build attribute array and create the code fragments
# ATTRIBUTE_ACCESSORS
tmpREP_FUNCTION = tmpUPROPERTY = tmpINIALIZE_ATTRIBUTES = tmpOnRepFUNCTIONS = tmpDOREP = tmpTagDef = tmpGAMEPLAYTAGSIMPL = ''

for i, val in enumerate(attributeList):
	# .h
	tmpREP_FUNCTION += '\t' + REP_FUNCTION.replace('$ATTRIBUTE$', val) + '\n\n'
	tmpUPROPERTY += '\t' + UPROPERTY.replace('$NAME$', AttributSetName).replace('$ATTRIBUTE$', val) + '\n\n'
	tmpTagDef += '\t' + GAMEPLAYTAGSDEF.replace('$NAME$', AttributSetName).replace('$ATTRIBUTE$', val) + '\n'
  # .cpp
	tmpINIALIZE_ATTRIBUTES += INIALIZE_ATTRIBUTES.replace('$ATTRIBUTE$', val) + ', '
	tmpOnRepFUNCTIONS += OnRepFUNCTIONS.replace('$ATTRIBUTE$', val).replace('$NAME$', AttributSetName) + '\n'
	tmpDOREP += '\t' + DOREP.replace('$NAME$', AttributSetName).replace('$ATTRIBUTE$', val) + '\n'
	tmpGAMEPLAYTAGSIMPL += '\t' + GAMEPLAYTAGSIMPL.replace('$NAME$', AttributSetName).replace('$ATTRIBUTE$', val) + '\n'
# reset from tmp vars
REP_FUNCTION = tmpREP_FUNCTION
UPROPERTY = tmpUPROPERTY
INIALIZE_ATTRIBUTES = tmpINIALIZE_ATTRIBUTES[:-2]
OnRepFUNCTIONS = tmpOnRepFUNCTIONS
DOREP = tmpDOREP
GAMEPLAYTAGSDEF = tmpTagDef
GAMEPLAYTAGSIMPL = tmpGAMEPLAYTAGSIMPL
# template for .h
content_h = """// Generated by Attribute Generator (https://bit.ly/AttributeGenerator). MIT license.
// Part of the series 'Create a game with Unreal Lyra - Developer Bastian - https://youtube.com/@BastianDev'
#pragma once

#include "AbilitySystemComponent.h"
#include "AbilitySystem/Attributes/LyraAttributeSet.h"
#include "AbilitySystem/LyraAbilitySystemComponent.h"
#include "NativeGameplayTags.h"
#include "$NAME$.generated.h"

class UObject;
struct FFrame;
$BASECLASS_LOGGING_H$
#pragma region GameplayTags
$GAMEPLAYTAGSDEF$
#pragma endregion

/**
 * $NAME$
 * $COMMENT$
 */
UCLASS(BlueprintType)
class U$NAME$ : public UGameAttributeSet
{
	GENERATED_BODY()

public:
	U$NAME$();

#pragma region GETTER-SETTER-INITS
$GETSETINIT$
#pragma endregion

protected:
#pragma region Replication
$REP_FUNCTION$
#pragma endregion

#pragma region Event pre and post Gameplay Effects
	// events pre or post applying a Gameplay Effect
	virtual void PostGameplayEffectExecute(const FGameplayEffectModCallbackData& Data) override;
	// events pre or post applying an Attribute change
	virtual void PreAttributeBaseChange(const FGameplayAttribute& Attribute, float& NewValue) const override;
	virtual void PreAttributeChange(const FGameplayAttribute& Attribute, float& NewValue) override;
	virtual void PostAttributeChange(const FGameplayAttribute& Attribute, float OldValue, float NewValue) override;
	virtual void OnAttributeAggregatorCreated(const FGameplayAttribute& Attribute, FAggregator* NewAggregator) const override;
#pragma endregion

	// helper function to make sure that the attribute stays between zero and max
	void ClampAttribute(const FGameplayAttribute& Attribute, float& NewValue) const;

private:

	// one property per attribute
$UPROPERTY$
};"""

# template for .cpp
content_cpp = """// Generated by Attribute Generator (https://bit.ly/AttributeGenerator). MIT license.
// Part of the series 'Create a game with Unreal Lyra - Developer Bastian - https://youtube.com/@BastianDev'

#include "$NAME$.h"
#include "AbilitySystem/Attributes/LyraAttributeSet.h"
#include "AbilitySystemGlobals.h"
#include "Net/UnrealNetwork.h"
#include "LyraGameplayTags.h"
#include "AbilitySystem/LyraAbilitySystemComponent.h"
#include "Engine/World.h"
#include "GameplayEffectExtension.h"
#include "Messages/LyraVerbMessage.h"
#include "Messages/LyraNotificationMessage.h"
#include "GameFramework\GameplayMessageSubsystem.h"

#include UE_INLINE_GENERATED_CPP_BY_NAME($NAME$)
$BASECLASS_LOGGING_CPP$
#pragma region GameplayTags
$GAMEPLAYTAGSIMPL$
#pragma endregion

class FLifetimeProperty;

U$NAME$::U$NAME$()
	: $INIALIZE_ATTRIBUTES$
{
}

#pragma region Replication
void U$NAME$::GetLifetimeReplicatedProps(TArray&lt;FLifetimeProperty&gt;& OutLifetimeProps) const
{
	Super::GetLifetimeReplicatedProps(OutLifetimeProps);
$DOREP$
}

$OnRepFUNCTIONS$
#pragma endregion

#pragma region PRE-POST Gameplay Effects
void U$NAME$::PostGameplayEffectExecute(const FGameplayEffectModCallbackData& Data)
{
	Super::PostGameplayEffectExecute(Data);
	if (Data.EvaluatedData.Magnitude != 0.0f)
	{
		FGameplayTag Tag = FGameplayTag::RequestGameplayTag(FName("AttributeSets.$NAME$." + Data.EvaluatedData.Attribute.AttributeName));
		if (Tag.IsValid())
		{
			FLyraVerbMessage Message;
			Message.Verb = Tag;
			Message.Instigator = Data.EffectSpec.GetEffectContext().GetEffectCauser();
			Message.InstigatorTags = *Data.EffectSpec.CapturedSourceTags.GetAggregatedTags();
			ULyraAbilitySystemComponent* ASC = CastChecked&lt;ULyraAbilitySystemComponent&gt;(GetOuter(), ECastCheckedType::NullAllowed);
			if (IsValid(ASC))
			{
				Message.Target = ASC->GetOwner();
			}
			Message.TargetTags = *Data.EffectSpec.CapturedTargetTags.GetAggregatedTags();
			Message.Magnitude = Data.EvaluatedData.Magnitude;

			UGameplayMessageSubsystem& MessageSystem = UGameplayMessageSubsystem::Get(GetWorld());
			MessageSystem.BroadcastMessage(Tag, Message);
			UE_LOG(GameAttributeSet, Verbose, TEXT("PostGameplayEffectExecute: Broadcasted msg for tag '%s' with insigator '%s' and target '%s'")
				, Tag.GetTagName(), *Message.Instigator->GetName(), *Message.Target->GetName());
		}
		else
		{
			UE_LOG(GameAttributeSet, Error, TEXT("PostGameplayEffectExecute: Broadcast failed, tag '%s' is not valid"), Tag.GetTagName());
		}
	}
	else
	{
		UE_LOG(GameAttributeSet, VeryVerbose, TEXT("PostAttributeChange: No message was broadcasted, magnitude = 0"));
	}
}
#pragma endregion

#pragma region PRE-POST Attribute Changes

void U$NAME$::PreAttributeBaseChange(const FGameplayAttribute& Attribute, float& NewValue) const
{
	Super::PreAttributeBaseChange(Attribute, NewValue);
	ClampAttribute(Attribute, NewValue);
}

void U$NAME$::PreAttributeChange(const FGameplayAttribute& Attribute, float& NewValue)
{
	Super::PreAttributeChange(Attribute, NewValue);
	ClampAttribute(Attribute, NewValue);
}

void U$NAME$::PostAttributeChange(const FGameplayAttribute& Attribute, float OldValue, float NewValue)
{
	Super::PostAttributeChange(Attribute, OldValue, NewValue);
	ULyraAbilitySystemComponent* ASC = CastChecked&lt;ULyraAbilitySystemComponent&gt;(GetOuter(), ECastCheckedType::NullAllowed);
	if (IsValid(ASC))
	{
		if (Attribute.GetGameplayAttributeData(this)->GetCurrentValue() > Attribute.GetGameplayAttributeData(this)->GetBaseValue())
		{
			ASC->ApplyModToAttribute(Attribute, EGameplayModOp::Override, NewValue);
		}
		// cast a message with PlayerState and Tag Channel for everyone listening
		if (IsValid(ASC->GetOwner()))
		{
			FGameplayTag Tag = FGameplayTag::RequestGameplayTag(FName("AttributeSets.$NAME$." + Attribute.AttributeName));
			if (Tag.IsValid())
			{
				FLyraNotificationMessage Message;
				Message.TargetChannel = Tag;
				Message.TargetPlayer = (APlayerState*)ASC->GetOwner();
				UGameplayMessageSubsystem& MessageSystem = UGameplayMessageSubsystem::Get(GetWorld());
				MessageSystem.BroadcastMessage(Tag, Message);
				UE_LOG(GameAttributeSet, Verbose, TEXT("PostAttributeChange: Broadcasted msg for tag '%s' - oldvalue: %d, new value: %d")
					, Tag.GetTagName(), OldValue, NewValue);
			}
			else
			{
				UE_LOG(GameAttributeSet, Error, TEXT("PostAttributeChange: Broadcast failed, tag '%s' is not valid"), Tag.GetTagName());
			}
		}
		else
		{
			UE_LOG(GameAttributeSet, Error, TEXT("PostAttributeChange: Getting a valid owner from ASC failed - aborted"));
		}
	}
	else
	{
		UE_LOG(GameAttributeSet, Error, TEXT("PostAttributeChange: Getting a valid ASC failed - aborted"));
	}
}

void U$NAME$::OnAttributeAggregatorCreated(const FGameplayAttribute& Attribute, FAggregator* NewAggregator) const
{
	Super::OnAttributeAggregatorCreated(Attribute, NewAggregator);

	if (!NewAggregator)
	{
		return;
	}
	/*
	// Example from Tranek: Only leave the most negative mod but all positive mods on each attribute (example: movement speed)
	if (Attribute == GetMoveSpeedAttribute())
	{
		NewAggregator->EvaluationMetaData = &FAggregatorEvaluateMetaDataLibrary::MostNegativeMod_AllPositiveMods;
	}
	*/
}

void U$NAME$::ClampAttribute(const FGameplayAttribute& Attribute, float& NewValue) const
{
	// Do not allow the new value to be higher than base value
	auto Data = Attribute.GetGameplayAttributeData(this);
	if (Data->GetBaseValue() < Data->GetCurrentValue())
	{
		NewValue = Data->GetBaseValue();
		UE_LOG(GameAttributeSet, Warning, TEXT("Current value for %s is %d, but base value is smaller at %d. New value clamped at to base value")
			, *Attribute.AttributeName, Data->GetCurrentValue(), Data->GetBaseValue());
	}
}
#pragma endregion
"""
# template for GET-SET-INIT
GetSetInit = """
#pragma region GET-SET-INIT $ATTRIBUTE$

	/** Gets current $ATTRIBUTE$'s FGameplayAttribute */
	UFUNCTION(BlueprintPure, Category = "$NAME$|$ATTRIBUTE$")
	static FGameplayAttribute Get$ATTRIBUTE$Attribute()
	{
		static FProperty* Prop = FindFieldChecked&lt;FProperty&gt;(U$NAME$::StaticClass(), GET_MEMBER_NAME_CHECKED(U$NAME$, $ATTRIBUTE$));
		UE_LOG(GameAttributeSet, VeryVerbose, TEXT("Ge|$ATTRIBUTE$Attribute: Returned |$ATTRIBUTE$ FProperty '%s'"), *GetNameSafe(Prop));
		return Prop;
	}

	/** Gets current $ATTRIBUTE$ base value (without any currently ongoing modifications */
	UFUNCTION(BlueprintPure, Category = "$NAME$|$ATTRIBUTE$")
		FORCEINLINE float GetBase$ATTRIBUTE$() const
	{
		UE_LOG(GameAttributeSet, VeryVerbose, TEXT("Get$ATTRIBUTE$Attribute: Returned $ATTRIBUTE$ base value: '%d'"), $ATTRIBUTE$.GetBaseValue());
		return $ATTRIBUTE$.GetBaseValue();
	}

	/** Gets current $ATTRIBUTE$ value. Current value is base value plus/minus all modifiers */
	UFUNCTION(BlueprintPure, Category = "$NAME$|$ATTRIBUTE$")
	FORCEINLINE float GetCurrent$ATTRIBUTE$() const
	{
		UE_LOG(GameAttributeSet, VeryVerbose, TEXT("Get$ATTRIBUTE$Attribute: Returned $ATTRIBUTE$ current value: '%d'"), $ATTRIBUTE$.GetCurrentValue());
		return $ATTRIBUTE$.GetCurrentValue();
	}

	/** Gets relative $ATTRIBUTE$ value between 0 and 1. Current value is base value plus/minus all modifiers */
	UFUNCTION(BlueprintPure, Category = "$NAME$|$ATTRIBUTE$")
		FORCEINLINE float GetRelative$ATTRIBUTE$() const
	{
		UE_LOG(GameAttributeSet, VeryVerbose, TEXT("Get$ATTRIBUTE$Attribute: Returned $ATTRIBUTE$ relative value: '%d'")
			, $ATTRIBUTE$.GetCurrentValue() / $ATTRIBUTE$.GetBaseValue());
		return $ATTRIBUTE$.GetCurrentValue() / $ATTRIBUTE$.GetBaseValue();
	}

	/** Sets $ATTRIBUTE$ base (and current) value. Base value is the permanent base, current value is after all modifiers. Should only be called on initialization */
	UFUNCTION(BlueprintCallable, BlueprintAuthorityOnly, Category = "$NAME$|$ATTRIBUTE$")
		FORCEINLINE void SetBase$ATTRIBUTE$(float NewVal)
	{
		UAbilitySystemComponent* AbilityComp = GetOwningAbilitySystemComponent();
		if (ensure(AbilityComp))
		{
			AbilityComp->SetNumericAttributeBase(Get$ATTRIBUTE$Attribute(), NewVal);
		}
		else
		{
			UE_LOG(GameAttributeSet, Error, TEXT("SetBase$ATTRIBUTE$: Failed to find valid ASC"));
		}
	}

	/** Each attribute corresponds to a FText to be localized */
	const FText Attributes_$NAME$_$ATTRIBUTE$_DisplayName = NSLOCTEXT("$NAME$","Attributes_$NAME$_$ATTRIBUTE$","$ATTRIBUTE$");

	/** Each attribute corresponds to a gameplay tag with the format AttributeSets.YourAttributeSetName.AttributeName */
	UFUNCTION(BlueprintPure, Category = "$NAME$|$ATTRIBUTE$")
	FORCEINLINE FGameplayTag GetGameplayTag_$ATTRIBUTE$() const;
#pragma endregion
"""
tmpGetSetInit = ''
for i, val in enumerate(attributeList):
  tmpGetSetInit += '\t' + GetSetInit.replace('$ATTRIBUTE$', val).replace('$NAME$', AttributSetName) + '\n'
GetSetInit = tmpGetSetInit
# process .h - replace all variables
content_h = content_h.replace('$NAME$', AttributSetName).replace('$BASECLASS_LOGGING_H$', BASECLASS_LOGGING_H).replace('$GAMEPLAYTAGSDEF$', GAMEPLAYTAGSDEF).replace('$COMMENT$', AttributSetComment).replace('$REP_FUNCTION$', REP_FUNCTION).replace('$UPROPERTY$', UPROPERTY).replace('$GETSETINIT$', GetSetInit)
# process .cpp - replace all variables
content_cpp = content_cpp.replace('$NAME$', AttributSetName).replace('$GAMEPLAYTAGSIMPL$', GAMEPLAYTAGSIMPL).replace('$BASECLASS_LOGGING_CPP$', BASECLASS_LOGGING_CPP).replace('$INIALIZE_ATTRIBUTES$', INIALIZE_ATTRIBUTES).replace('$DOREP$', DOREP).replace('$OnRepFUNCTIONS$', OnRepFUNCTIONS)

# build up display html code
#@title
from IPython.display import HTML, Javascript, display
from google.colab import output
# setup html
textToDisplay =  "<style>.flex-parent-element {  display: flex;  width: 50%; maxHeight: 100px; overflow: scroll;} .flex-child-element {  flex: 1;  border: 2px solid blueviolet;  margin: 10px;  maxHeight: 100px; overflow: scroll;} .flex-child-element:first-child {   margin-right: 20px;}"
textToDisplay += ".p_block {font-weight: normal; margin-left: 25px; line-height:150%; background-color: #fffff0;} .btn { border:0.1em solid rgba(255,255,255,0); } .btn:hover { border-color: #3366ff; } .btn_share { border:0.1em solid rgba(255,255,255,0);} .btn_share:hover { border-color: #3366ff; }"
textToDisplay += "span {font-weight: bold;}</style>"
textToDisplay += "<div class=\"flex-parent-element\"><div class=\"flex-child-element magenta\"><span>Content .h file for '" + AttributSetName + "':&nbsp;</span><button id='b_h_content' class='btn' data-clipboard-action='copy' data-clipboard-target='#h_content'><img src='https://cdnjs.cloudflare.com/ajax/libs/octicons/8.5.0/svg/clippy.svg' alt='Copy to clipboard' /></button><pre><p class='p_block' id='h_content'><code>" + content_h.replace('\n', '<br />') + "</code></p></pre></div>"
textToDisplay += "<div class=\"flex-child-element green\"><span>Content .cpp file for '" + AttributSetName + "':&nbsp;</span><button id='b_cpp_content' class='btn' data-clipboard-action='copy' data-clipboard-target='#cpp_content'><img src='https://cdnjs.cloudflare.com/ajax/libs/octicons/8.5.0/svg/clippy.svg' alt='Copy to clipboard' /></button><pre><p class='p_block' id='cpp_content'><code>" + content_cpp.replace('\n', '<br />') + "</code></p></pre></div></div>"
textToDisplay += "<script src='https://cdnjs.cloudflare.com/ajax/libs/clipboard.js/2.0.11/clipboard.min.js'></script><script>var clipboard = new ClipboardJS('.btn');</script>"
HTML(textToDisplay)